# Aritmética de punto flotante

## Notación científica

$$
32487.23749 = 3.248723749 \times 10^4 \\
0.000892 = 8.92 \times 10^{-4}
$$

## Normalización

Sirve para mantener la uniformidad de la notación científica.

### Explicita

El punto decimal se recorre a la izquierda de la cifra mas significativa. Ejemplo:

$$ 32487.23749 = 0.3248723749 \times 10^5 $$

### Implícita

El punto decimal se recorre a la derecha de la cifra mas significativa. Ejemplo:

$$ 32487.23749 = 3.248723749 \times 10^5 $$

## Parte fraccionaria de un numero binario

Asi como los números en base 10 los números en base 2 también puede tener una parte fraccionaria, primero hay que recordar como se forma un numero entero en sistema binario y decimal.

Sea $ c_i $ la cifra en la posición $ i $ y $ n $ el máximo numero de cifras.

$$ 
\textrm{binario: } 2^n(c_n) + 2^{n-1}(c_{n-1}) + \cdots + 2^2(c_2) + 2^1(c_1) + 2^0(c_1) \\ 
\textrm{decimal: } 10^n(c_n) + 10^{n-1}(c_{n-1}) + \cdots + 10^2(c_2) + 10^1(c_1) + 10^0(c_1)
$$

Recordando la parte fraccionaria de un numero decimal tenemos que $ 0.1 = \frac{1}{10} $, $ 0.01 = \frac{1}{10^2} $, $ 0.001 = \frac{1}{10^3} $ y asi sucesivamente. Asi tenemos que:

$$ \textrm{decimal fraccionario: } 10^{-1}(c_1) + 10^{-2}(c_2) + 10^{-3}(c_2) + \cdots + 10^{-(n-1)}(c_{n-1}) + 10^{-n}(c_n) $$ 

Asi mismo funciona para el sistema binario:

$$ \textrm{binario fraccionario: } 2^{-1}(c_1) + 2^{-2}(c_2) + 2^{-3}(c_2) + \cdots + 2^{-(n-1)}(c_{n-1}) + 2^{-n}(c_n) $$ 

Ahora por ejemplo si quisiéramos transforma el siguiente numero $1011.101_2$ a decimal, usando la expresiones anteriores tenemos que:

$$
2^4(1) + 2^2(0) + 2^1(1) + 2^0(1) + 2^{-1}(1) + 2^{-2}(0) + 2^{-3}(1) \\
= 16 + 0 + 2 + 1 + \frac{1}{2} + \frac{0}{4} + \frac{1}{8} \\
= 19 + \frac{5}{8} \\
= 19.625
$$ 

## Representación de números de punto flotante en sistemas computacionales

### Antiguamente

De las primeras formas de representar un numero de punto flotante fue hacerlo de la siguiente forma $[b, n]$ donde $b$ es el numero en binario y $n$ son las posiciones que se recorrerá el punto tomando en cuenta que el punto se considera que por defecto esta en el final del numero, por ejemplo:

$$
\textrm{Un numero de 8 bits: }[00010101,2] \\
\textrm{Primero recorremos el punto en el numero base 2: } 101.01\\
\textrm{Finalmente transformamos el numero al sistema decimal: } 5.25\\
$$

### En la actualidad

Para representar un numero de punto flotante se pueden utilizar distintas cantidades de bytes que se dividen en 3 sectores de bits que tienen distintos significados.

|#bytes|Nombre común|Signo|Exponente|Fracción (mantissa)|
|:---:|:---:|:---:|:---:|:---:|
|1|(didáctico)|1|3|4|
|2|half precision|1|5|10|
|4|single precision|1|8|23|
|8|double precision|1|11|52|

Para calcular el valor del numero se utiliza la siguiente formula:

$$ Valor = S \cdot 2^E \cdot 1.M $$


### Signo ($S$)

El signo siempre es representado por el primer bit, para calcular su valor se utiliza la siguiente expresión:

Sea $b_i$ el valor del bit en la posición $i$ y $S$ el valor del signo.

$$ S = (-1)^{b_0} $$

### Exponente ($E$)

El exponente sera el encargado de recorrer las posiciones del punto decimal de nuestra fracción, para el caso de esta representación es necesario poder recorrer el punto hacia delante y haca atrás, asi que el exponente debe poder tener valores negativos, para esto debemos tener un valor auxiliar llamado **bias** ($b$) que nos permitirá representar un numero negativo y se calcula la siguiente manera:

Sea $k$ la cantidad de bits correspondientes al exponente

$$ b = 2^{k - 1} - 1 $$

El valor de $E$ se calcula de la siguiente manera:

Sea $e$ el valor en decimal de los bits correspondientes al exponente

$$ E = e - b $$

### Fracción ($M$)

La fracción o mantissa se le conoce también como la cifra significativa, esta se obtiene igual que la parte fraccionaria de un numero binario y siempre se le suma una unidad entera, como si esta fuera normalizada de forma implícita

In [216]:
from bitarray import bitarray
import re

## Definición de las funciones

### `fracdec_fracbin`

Recibe un numero decimal con parte fraccionaria en formato string y lo transforma a un numero binario con parte decimal.


In [217]:
def entero_bin(entero: int) -> bitarray:
    bin = bitarray()
    if entero <= 0:
        return bitarray()
    while entero > 0:
        bin.append(entero % 2)
        entero //= 2
    return bin[-1::-1]


def num_cifras(entero: int) -> int:
    cifras = 0
    while entero > 0:
        cifras += 1
        entero //= 10
    return cifras


def fraccion_bin(fraccion: int, cifras:int, limite: int = 150) -> bitarray:
    bin = bitarray()
    divisor = 10 ** cifras
    if fraccion <= 0:
        return bitarray()
    while fraccion > 0 and len(bin) < limite:
        fraccion *= 2
        if fraccion >= divisor:
            bin.append(1)
            fraccion %= divisor
        else:
            bin.append(0)
    return bin


def fracdec_fracbin(numero: str):
    if '.' not in numero:
        numero.append('.')

    if numero[0] == '+':
        signo = bitarray([0])
        numero.pop(0)
    elif numero[0] == '-':
        signo = bitarray([1])
        numero.pop(0)
    else:
        signo = bitarray([0])

    entero, fraccion = numero.split('.')
    if entero == '':
        entero = '0'
    if fraccion == '':
        fraccion = '0'

    if entero == '0' and fraccion == '0':
        return signo + bitarray([0] * 31)

    return (
        signo,
        entero_bin(int(entero)),
        fraccion_bin(int(fraccion), len(fraccion))
    )

### `fracbin_floatbin`

Recibe un numero binario con punto decimal en y lo transforma a un arreglo de bits con formato float.


In [218]:
def fracbin_floatbin(signo:bitarray, entero: bitarray, fraccion: bitarray, type: int = 32) -> bitarray:
    mantissa_size = 23
    exponent_size = 8
    bias = 127

    mantissa = entero + fraccion
    ceros = 0
    while mantissa[0] == 0:
        ceros += 1
        mantissa.pop(0)
    mantissa = mantissa + bitarray('0' * (mantissa_size + 1))
    mantissa = mantissa[1: 23 + 1]

    exp_real = int()
    if len(entero) >= 1:
        exponente = entero_bin(bias + len(entero) - 1)
    else:
        exponente = entero_bin(bias - (ceros + 1))
    exponente = bitarray('0' * (exponent_size - len(exponente))) + exponente

    # print(signo, exponente, mantissa, sep= '\n')
    return signo + exponente + mantissa

In [219]:
flotante = fracbin_floatbin(*fracdec_fracbin('25.2525'))
flotante

bitarray('01000001110010100000010100011110')

### `floatbin_fracbin`

Recibe un arreglo binario con formato float y lo transforma en un numero binario con punto decimal.


In [220]:
def floatbin_fracbin(flotante: bitarray, type: int = 32):
    bias = 127
    exponente_size = 8
    signo = flotante[0]
    exponente = bin_entero(flotante[1: 1 + exponente_size]) - bias

    mantissa = flotante[1 + exponente_size:]
    if exponente >= 0:
        entero = bitarray([1]) + mantissa[:exponente]
        fraccion =  mantissa[exponente:]
    else:
        entero = bitarray()
        fraccion = bitarray([0] * abs(exponente)) + bitarray([1]) + mantissa

    return signo, entero, fraccion


def bin_entero(bin: bitarray) -> int:
    potencia = len(bin) - 1
    res = 0
    for b in bin:
        res += b * (2 ** potencia)
        potencia -= 1
    return res


### `fracbin_fracdec`

Recibe un numero binario con punto decimal y lo transforma en uno decimal con parte fraccionaria.


In [221]:
def fracbin_fracdec(signo: bitarray, entero: bitarray, fraccion: bitarray) -> str:
    entero = str(bin_entero(entero))
    fraccion = fraccion_str(*bin_fraccion(fraccion))
    return entero + '.' + fraccion


def bin_fraccion(bin: bitarray):
    potencia = len(bin)
    numerador = 0
    denominador = 2 ** potencia
    while potencia:
        potencia -= 1
        numerador += bin[0] * 2 ** potencia
        bin.pop(0)
    return numerador, denominador


def fraccion_str(numerador: int, denominador: int) -> str:
    res = str()
    i = 0
    while numerador > 0 and i < 10:
        res = res + str(numerador // denominador)
        numerador = (numerador % denominador) * 10
        i += 1
    return res[1:]

In [222]:
fracbin_fracdec(*floatbin_fracbin(flotante))

'25.252498626'